In [9]:
import torch
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'matplotlib.backends.registry'

In [ ]:
url = r"C:\Users\Дарина\OneDrive\Робочий стіл\3 курс 1 семестр\Машинне\machine\household_power_consumption.csv"
df = pd.read_csv(url, sep=',', low_memory=False, parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, na_values=['nan', '?'])



df = df[['datetime', 'Voltage']].dropna()


scaler = MinMaxScaler(feature_range=(0, 1))
df['Voltage'] = scaler.fit_transform(df[['Voltage']])


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])


In [ ]:
X = df['Voltage'].values.reshape(-1, 1)
y = df['Voltage'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
from sklearn.metrics import r2_score


loss, mae = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")
print(f"Test R-squared (Accuracy): {r2}")


In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import matplotlib.dates as mdates

# Denormalize the actual and predicted voltage values
y_test_denorm = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_denorm = scaler.inverse_transform(y_pred)

# Extract the datetime values corresponding to the test data
time_test = df['datetime'].values[-len(X_test):]

# Plot the actual vs predicted values (denormalized)
plt.figure(figsize=(10, 6))
plt.plot(time_test, y_test_denorm, label='Actual Voltage')
plt.plot(time_test, y_pred_denorm, label='Predicted Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.legend()

# Format the x-axis to show datetime
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1000))  # Adjust interval as needed
plt.gcf().autofmt_xdate()  # Rotate and format x-axis labels

plt.show()


In [ ]:
import matplotlib.dates as mdates

# Select only the last 24 hours of data
# Assuming 'datetime' is already in datetime format
start_time = df['datetime'].max() - pd.Timedelta(days=1)  # 24 hours from the last timestamp
df_24_hours = df[df['datetime'] >= start_time]

# Rebuild X_test and y_test for the last 24 hours
X_test_24 = df_24_hours['Voltage'].values.reshape(-1, 1)
y_test_24 = df_24_hours['Voltage'].values

# Predict based on the last 24 hours of test data
y_pred_24 = model.predict(X_test_24)

# Denormalize the actual and predicted voltage values
y_test_24_denorm = scaler.inverse_transform(y_test_24.reshape(-1, 1))
y_pred_24_denorm = scaler.inverse_transform(y_pred_24)

# Extract the datetime values corresponding to the last 24 hours
time_test_24 = df_24_hours['datetime'].values

# Plot the actual vs predicted values (denormalized) for 24 hours
plt.figure(figsize=(10, 6))
plt.plot(time_test_24, y_test_24_denorm, label='Actual Voltage')
plt.plot(time_test_24, y_pred_24_denorm, label='Predicted Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.legend()

# Format the x-axis to show datetime for the last 24 hours
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))  # 1-hour intervals
plt.gcf().autofmt_xdate()  # Rotate and format x-axis labels

plt.show()
